In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests4

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn,return_model=True):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    if return_model:
        return ll1,grad1,hess1,model1_param,model1_deriv,ll2,grad2,hess2,model2_param,model2_deriv
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()

# Ex 0.5

In [3]:
beta = 0.0

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.94 0.06 0.  ] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.94 & 1.00 & 1.00 & 0.91   \\
Model 1 & 0.00 & 0.00 & 0.06 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.09   \\
\hline
\end{tabular}
llr:9.152040535081532, std:13.761738760837531
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.94, 0.06, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.91, 0.  , 0.09]), 9.152040535081532, 13.761738760837531, 17.476067954140508)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.89 0.11 0.  ] [0.99 0.01 0.  ] [0.99 0.01 0.  ]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.89 & 0.99 & 0.99 & 0.95   \\
Model 1 & 0.00 & 0.00 & 0.11 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.05   \\
\hline
\end{tabular}
llr:11.835050377155591, std:21.172980790270937
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.89, 0.11, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.95, 0.  , 0.05]), 11.835050377155591, 21.172980790270937, 19.16528222025136)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.91 0.09 0.  ] [0.98 0.02 0.  ] [0.98 0.02 0.  ]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.91 & 0.98 & 0.98 & 0.89   \\
Model 1 & 0.00 & 0.00 & 0.09 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.11   \\
\hline
\end{tabular}
llr:14.37687164801884, std:24.337354384070036
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.91, 0.09, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.89, 0.  , 0.11]), 14.37687164801884, 24.337354384070036, 21.451940248271914)


# Ex 0.5

In [7]:
beta = 0.5

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.97 0.03 0.  ] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 1.00 & 0.97 & 1.00 & 1.00 & 0.93   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.08 & 0.00 & 0.00 & 0.00 & 0.00 & 0.07   \\
\hline
\end{tabular}
llr:11.328687823075546, std:28.79232913740606
(array([0.92, 0.  , 0.08]), array([1., 0., 0.]), array([0.97, 0.03, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.93, 0.  , 0.07]), 11.328687823075546, 28.79232913740606, 29.95067487318527)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.96 0.04 0.  ] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 1.00 & 0.96 & 1.00 & 1.00 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.04 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.06 & 0.00 & 0.00 & 0.00 & 0.00 & 0.03   \\
\hline
\end{tabular}
llr:13.478810251497556, std:42.07973812027605
(array([0.94, 0.  , 0.06]), array([1., 0., 0.]), array([0.96, 0.04, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.97, 0.  , 0.03]), 13.478810251497556, 42.07973812027605, 43.81826295514443)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.94 0.04 0.02] [0.95 0.02 0.03] [0.95 0.02 0.03]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 1.00 & 0.94 & 0.95 & 0.95 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.04 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.06 & 0.00 & 0.02 & 0.03 & 0.03 & 0.03   \\
\hline
\end{tabular}
llr:11.379693642422327, std:53.00461430831428
(array([0.94, 0.  , 0.06]), array([1., 0., 0.]), array([0.94, 0.04, 0.02]), array([0.95, 0.02, 0.03]), array([0.95, 0.02, 0.03]), array([0.97, 0.  , 0.03]), 11.379693642422327, 53.00461430831428, 57.73890275407179)


# Ex 1

In [7]:
beta = 1

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.91 0.06 0.03] [0.91 0.05 0.04] [0.91 0.05 0.04]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 1.00 & 0.91 & 0.91 & 0.91 & 0.92   \\
Model 1 & 0.00 & 0.00 & 0.06 & 0.05 & 0.05 & 0.00   \\
Model 2 & 0.19 & 0.00 & 0.03 & 0.04 & 0.04 & 0.08   \\
\hline
\end{tabular}
llr:11.655447467694582, std:62.48478905063853
(array([0.81, 0.  , 0.19]), array([1., 0., 0.]), array([0.91, 0.06, 0.03]), array([0.91, 0.05, 0.04]), array([0.91, 0.05, 0.04]), array([0.92, 0.  , 0.08]), 11.655447467694582, 62.48478905063853, 49.099772390796296)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.93 0.04 0.03] [0.96 0.01 0.03] [0.96 0.01 0.03]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 1.00 & 0.93 & 0.96 & 0.96 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.04 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.06 & 0.00 & 0.03 & 0.03 & 0.03 & 0.01   \\
\hline
\end{tabular}
llr:17.008305585313, std:88.70426917806518
(array([0.94, 0.  , 0.06]), array([1., 0., 0.]), array([0.93, 0.04, 0.03]), array([0.96, 0.01, 0.03]), array([0.96, 0.01, 0.03]), array([0.99, 0.  , 0.01]), 17.008305585313, 88.70426917806518, 81.35467259605653)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.96 0.03 0.01] [0.96 0.03 0.01] [0.96 0.03 0.01]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.96 & 0.96 & 0.96 & 0.99   \\
Model 1 & 0.02 & 0.02 & 0.03 & 0.03 & 0.03 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
llr:33.44187798438217, std:120.61398540465537
(array([0.95, 0.02, 0.03]), array([0.95, 0.02, 0.03]), array([0.96, 0.03, 0.01]), array([0.96, 0.03, 0.01]), array([0.96, 0.03, 0.01]), array([0.99, 0.  , 0.01]), 33.44187798438217, 120.61398540465537, 118.32960007901883)


# Ex 1.5

In [11]:
beta = 1.5

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.93 0.02 0.05] [0.91 0.01 0.08] [0.91 0.01 0.08]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.91 & 0.99 & 0.93 & 0.91 & 0.91 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.09 & 0.01 & 0.05 & 0.08 & 0.08 & 0.02   \\
\hline
\end{tabular}
llr:9.22045086536592, std:76.69834717348968
(array([0.91, 0.  , 0.09]), array([0.99, 0.  , 0.01]), array([0.93, 0.02, 0.05]), array([0.91, 0.01, 0.08]), array([0.91, 0.01, 0.08]), array([0.98, 0.  , 0.02]), 9.22045086536592, 76.69834717348968, 75.33145952961402)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.93 0.05 0.02] [0.95 0.03 0.02] [0.95 0.03 0.02]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.93 & 0.95 & 0.95 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.05 & 0.03 & 0.03 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.02 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
llr:24.752598146137824, std:131.85939697981718
(array([0.96, 0.01, 0.03]), array([0.96, 0.01, 0.03]), array([0.93, 0.05, 0.02]), array([0.95, 0.03, 0.02]), array([0.95, 0.03, 0.02]), array([1., 0., 0.]), 24.752598146137824, 131.85939697981718, 123.58671316146952)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.97 0.02 0.01] [0.95 0.02 0.03] [0.95 0.02 0.03]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.95 & 0.95 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.01 & 0.03 & 0.03 & 0.00   \\
\hline
\end{tabular}
llr:21.31724800592139, std:157.6894122050766
(array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.97, 0.02, 0.01]), array([0.95, 0.02, 0.03]), array([0.95, 0.02, 0.03]), array([1., 0., 0.]), 21.31724800592139, 157.6894122050766, 170.3610984545461)


# Evidence of power...

In [5]:
def gen_data2(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([0,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

In [16]:
beta = 0.5

In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[1. 0. 0.] [1. 0. 0.] [1. 0. 0.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.78 & 1.00 & 1.00 & 1.00 & 1.00 & 0.82   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.22 & 0.00 & 0.00 & 0.00 & 0.00 & 0.18   \\
\hline
\end{tabular}
llr:-1.676616465551855, std:21.6659972385679
(array([0.78, 0.  , 0.22]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.82, 0.  , 0.18]), -1.676616465551855, 21.6659972385679, 19.158998213927433)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.99 0.   0.01] [0.99 0.   0.01] [0.99 0.   0.01]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.62 & 1.00 & 0.99 & 0.99 & 0.99 & 0.80   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.38 & 0.00 & 0.01 & 0.01 & 0.01 & 0.20   \\
\hline
\end{tabular}
llr:-19.09134224720412, std:18.56698319942619
(array([0.62, 0.  , 0.38]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), array([0.99, 0.  , 0.01]), array([0.99, 0.  , 0.01]), array([0.8, 0. , 0.2]), -19.09134224720412, 18.56698319942619, 21.903501524630546)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.83 0.   0.17] [0.63 0.   0.37] [0.63 0.   0.37]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.35 & 1.00 & 0.83 & 0.63 & 0.63 & 0.70   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.65 & 0.00 & 0.17 & 0.37 & 0.37 & 0.30   \\
\hline
\end{tabular}
llr:-50.31980755020021, std:19.744302295751364
(array([0.35, 0.  , 0.65]), array([1., 0., 0.]), array([0.83, 0.  , 0.17]), array([0.63, 0.  , 0.37]), array([0.63, 0.  , 0.37]), array([0.7, 0. , 0.3]), -50.31980755020021, 19.744302295751364, 24.598460754161625)


# Ex 1

In [20]:
beta = 1

In [21]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.92 0.   0.08] [0.86 0.   0.14] [0.86 0.   0.14]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.41 & 1.00 & 0.92 & 0.86 & 0.86 & 0.70   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.59 & 0.00 & 0.08 & 0.14 & 0.14 & 0.30   \\
\hline
\end{tabular}
llr:-37.857820229047746, std:25.514142855701998
(array([0.41, 0.  , 0.59]), array([1., 0., 0.]), array([0.92, 0.  , 0.08]), array([0.86, 0.  , 0.14]), array([0.86, 0.  , 0.14]), array([0.7, 0. , 0.3]), -37.857820229047746, 25.514142855701998, 24.21302024437084)


In [22]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.21 0.   0.79] [0.19 0.   0.81] [0.19 0.   0.81]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.10 & 1.00 & 0.21 & 0.19 & 0.19 & 0.31   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.90 & 0.00 & 0.79 & 0.81 & 0.81 & 0.69   \\
\hline
\end{tabular}
llr:-116.05432663002098, std:28.221653939499113
(array([0.1, 0. , 0.9]), array([1., 0., 0.]), array([0.21, 0.  , 0.79]), array([0.19, 0.  , 0.81]), array([0.19, 0.  , 0.81]), array([0.31, 0.  , 0.69]), -116.05432663002098, 28.221653939499113, 27.3081725933942)


In [23]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.02 0.   0.98] [0.02 0.   0.98] [0.02 0.   0.98]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.89 & 0.02 & 0.02 & 0.02 & 0.13   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.11 & 0.98 & 0.98 & 0.98 & 0.87   \\
\hline
\end{tabular}
llr:-238.36841831910385, std:31.526468450357264
(array([0., 0., 1.]), array([0.89, 0.  , 0.11]), array([0.02, 0.  , 0.98]), array([0.02, 0.  , 0.98]), array([0.02, 0.  , 0.98]), array([0.13, 0.  , 0.87]), -238.36841831910385, 31.526468450357264, 32.578987146910514)


# more...

In [6]:
beta = 1.5

In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.34 0.   0.66] [0.28 0.   0.72] [0.28 0.   0.72]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.16 & 0.99 & 0.34 & 0.28 & 0.28 & 0.42   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.84 & 0.01 & 0.66 & 0.72 & 0.72 & 0.58   \\
\hline
\end{tabular}
llr:-100.27150407363311, std:25.00261481128177
(array([0.16, 0.  , 0.84]), array([0.99, 0.  , 0.01]), array([0.34, 0.  , 0.66]), array([0.28, 0.  , 0.72]), array([0.28, 0.  , 0.72]), array([0.42, 0.  , 0.58]), -100.27150407363311, 25.00261481128177, 26.241800961241438)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0.01 0.   0.99] [0.01 0.   0.99] [0.01 0.   0.99]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.91 & 0.01 & 0.01 & 0.01 & 0.10   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.09 & 0.99 & 0.99 & 0.99 & 0.90   \\
\hline
\end{tabular}
llr:-265.041540504786, std:25.536501123166914
(array([0., 0., 1.]), array([0.91, 0.  , 0.09]), array([0.01, 0.  , 0.99]), array([0.01, 0.  , 0.99]), array([0.01, 0.  , 0.99]), array([0.1, 0. , 0.9]), -265.041540504786, 25.536501123166914, 32.59128941386266)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, beta =beta)
mc_out = vuong_tests4.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests4.print_mc(mc_out)
print(mc_out)

[0. 0. 1.] [0. 0. 1.] [0. 0. 1.]
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.88 & 0.00 & 0.00 & 0.00 & 0.01   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.12 & 1.00 & 1.00 & 1.00 & 0.99   \\
\hline
\end{tabular}
llr:-552.707183082735, std:39.214801181147365
(array([0., 0., 1.]), array([0.88, 0.  , 0.12]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0.01, 0.  , 0.99]), -552.707183082735, 39.214801181147365, 40.94219557568034)
